In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv1D, Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Multiply, Dense
from keras import backend as K

import tensorflow as tf
import cv2

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as K
import tensorflow.keras.models as KM
from keras.layers import Concatenate

def sa_block(inputs):
    kernel_size = 7
    avg_pool = tf.reduce_mean(inputs, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(inputs, axis=-1, keepdims=True)
    x = Concatenate()([avg_pool, max_pool])
    x = Conv2D(1, kernel_size, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(x)
    outputs = Multiply()([inputs, x])
    return outputs

In [ ]:
vgg16_model= vgg16 = tf.keras.applications.VGG16(include_top=False, weights="imagenet")
vgg16_model.summary()

In [ ]:
inputs       = vgg16_model.get_layer(index=0).output
block1_conv1 = vgg16_model.get_layer(name='block1_conv1') (inputs)
eca_out      = sa_block(block1_conv1)
block1_conv2 = vgg16_model.get_layer(name='block1_conv2') (eca_out)
outputs = Dense(units=1, activation='sigmoid') (block1_conv2)
custom_model = Model(inputs=[inputs], outputs = [outputs])
custom_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   1792        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 tf.math.reduce_mean_1 (TFOpLam  (None, None, None,   0          ['block1_conv1[2][0]']           
 bda)                           1)                                                            